In [ ]:
import sys,getopt
import os
import numpy as np
import pandas
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
### open file / load data

data_dir = "/Volumes/Diamond/Projects/Purification/Setup/IntStability/16_05_31_InterferometricStability"
file_name = "16_05_31_InterferometricStability.csv"
filename = os.path.join(data_dir,file_name)

scope = 'Slow' #TODO Get better scope names

if scope == 'Fast':
    # To open files from the fast scope (CSV)

    file = open(filename)
    for x in range(9):
            line = file.readline()
            if "HResolution" in line:
                TimeRes = float(line[-18:-1])
    file.close()
    
    subdiv = 1            
    data = pandas.read_csv(os.path.join(data_dir,filename),header=None,skiprows = 15)
    data = data.drop([0,2],1)
    data = data.rolling(window = 100).mean()
    vals = data.as_matrix()[100:-100:subdiv]
    del data
    
    TimeRes = TimeRes*subdiv
    time_points = TimeRes*np.arange(len(vals))
    
elif scope == 'Slow':
    data = pandas.read_csv(os.path.join(data_dir,filename),header=0,skiprows = 0,nrows=1)
    TimeRes = data['Increment'][0]

    subdiv = 1            
    data = pandas.read_csv(os.path.join(data_dir,filename),header=None,skiprows = 2)
    data = data.drop([0,2],1)
    data = data.rolling(window = 4).mean()
    vals = data.as_matrix()[10:-10:subdiv]
    del data
    TimeRes = TimeRes*subdiv
    time_points = TimeRes*np.arange(len(vals))

elif scope == 'LeCroy':
    data = pandas.read_csv(os.path.join(data_dir,filename),header=None,skiprows = 5,nrows=1)
    
    TimeRes = data[1][0]

    subdiv = 1            
    data = pandas.read_csv(os.path.join(data_dir,filename),header=None,skiprows = 30)
    data = data.drop([0,1,2],1)
    #data = data.rolling(window = 100).mean()
    vals = data.as_matrix()[0:-1:subdiv]
    del data
    TimeRes = TimeRes*subdiv
    time_points = TimeRes*np.arange(len(vals))
    

In [ ]:
# show data
start = 0.04
length = 0.05
show_all = True
fig = plt.figure()
ax = plt.subplot()
plt.plot(time_points,vals)
plt.title(file_name)
plt.xlabel('time (s)')
plt.ylabel('signal')
if not show_all:
    plt.xlim([start,start+length])
plt.show()

plt.close('all')

In [ ]:
# Phase deviation after time x (a.k.a. a slice of the allan deviation.)

#rescale the data to [0;+1] and take the arccos of those.

NormMax = np.amax(vals); NormMin = np.amin(vals)
print NormMax
print NormMin

valsRescaled = (np.array(vals)-NormMin)/(NormMax-NormMin)

#take arccos and the sqrt (we measure intensities not amplitudes!) to get the phases out and normalize by pi
fig = plt.figure()
ax = plt.subplot(2,1,1)
plt.plot(time_points,valsRescaled)
plt.ylabel(r"$\cos(\theta/2)^2$")
plt.title(file_name)
valsRescaled = 2 *np.arccos(valsRescaled)
ax = plt.subplot(2,1,2)
plt.plot(time_points,valsRescaled)
plt.ylabel(r"$\theta/\pi$")
plt.show()
plt.close('all')

In [ ]:
### check the deviation after x time
deltaT = 0.0025 # in seconds
deltaSteps = int(deltaT/TimeRes)
deviations = []

for i,v in enumerate(valsRescaled):
    
    if i+deltaSteps+1 > len(valsRescaled):
        break
    deviations.append(v-valsRescaled[i+deltaSteps])
    
fig = plt.figure()
ax = plt.subplot()

n, bins, patches = plt.hist(np.array(deviations),21,facecolor = 'g', edgecolor = 'None',normed=False)#,range = [-1,1])
plt.title(file_name)
plt.xlabel("Phase deviation after {:.1f} ms (pi)".format(deltaT*1e3))
plt.ylabel("Est. probability")
plt.show()
plt.close("all")

In [ ]:
sys.path.append(r'//Users/humphreys/Repositories/')

from analysis.lib.fitting import fit, common
from analysis.lib.tools import plot
fig = plt.figure()
ax = plt.subplot()
p0, fitfunc, fitfunc_str = common.fit_gauss(0.0, 1, 0,0.2)
fit_result = fit.fit1d(bins[:-1],n, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=[0])

plot.plot_fit1d(fit_result, np.linspace(-0.5,0.5,201), ax=ax, plot_data=True,color = 'r',add_txt = True, lw = 1)
plt.title(file_name)
plt.xlabel("Phase deviation after {:.1f} ms (pi)".format(deltaT*1e3))
plt.ylabel("occurences")
plt.show()
plt.close('all')

In [ ]:
### Plot the deviation as a function of time

deltaT = np.arange(0,20e-3,2e-5)
deltaSteps = (deltaT/TimeRes).astype(int)
std =  np.zeros(len(deltaSteps))

debug = False

for x,delSteps in enumerate(deltaSteps):
    
    if delSteps == 0:
        std[x] = 0
    else:
        deviations = (valsRescaled[delSteps:] - valsRescaled[:-delSteps])
        std[x] = np.std(deviations)
    
    
    if debug:
        fig = plt.figure()
        ax = plt.subplot()
        plt.plot(bin_edges[:-1],bins)
        plt.scatter([bin_edges[ind_left],bin_edges[ind_right]],[np.max(bins)/2,np.max(bins)/2])
        plt.show()
        plt.close('all')
    
#FWHM = FWHM/2.355 #FWHM to standard deviation

fig = plt.figure()
ax = plt.subplot()
plt.plot(deltaT*1e3, std)
plt.title(file_name)
plt.ylim([0, 1.1*plt.ylim()[1]])
plt.xlim([-0.5,plt.xlim()[1]])
plt.xlabel(r'Time difference (ms)')
plt.ylabel(r'Standard deviation (rad)')
plt.savefig(os.path.join(data_dir,'allanDev.pdf'))

plt.show()
#plt.close('all')

In [ ]:
fig = plt.figure()
ax = plt.subplot()
plt.plot(deltaT*1e3, std)
plt.title(file_name)
plt.ylim([0, 1.1*plt.ylim()[1]])
plt.xlim([-0.5,plt.xlim()[1]])
plt.xlabel(r'Time difference (ms)')
plt.ylabel(r'Standard deviation (rad)')
plt.savefig(os.path.join(data_dir,'allanDev.pdf'))

plt.show()

Surprising periodicity! Lets do a quick FFT to pull the frequencies

In [ ]:
### fast fourier transform
from scipy.fftpack import fft

fft = fft(FWHM,axis=0)
freq = np.fft.fftfreq(deltaT.shape[-1],deltaT[1]-deltaT[0])
N = len(fft)
fft = np.abs(fft[0:N/2])
freq = freq[0:N/2]

fig = plt.figure()
ax = plt.subplot()
        
plt.plot(freq,fft)
plt.xlim([20,1000])
plt.ylim([0,10])


from detect_peaks import detect_peaks

peaks = detect_peaks(np.squeeze(fft),mpd=3/(freq[1]-freq[0]))
peaks = peaks[np.argsort(np.squeeze(fft[peaks]))][::-1]
peaks = peaks[0:10]

for fre,ff in zip(freq[peaks], fft[peaks]):
    print 'freq: %d, height: %.4f' % (fre, ff)
    
plt.scatter(freq[peaks],fft[peaks],c='r')
plt.title(file_name)
plt.show()
plt.close(fig)